# Imports

In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
import pathlib
from pathlib import Path

pd.set_option('display.max_columns', None)

In [24]:
df = pd.read_parquet("./dados_violencia_mulheres_ses_concat_limpo.parquet")
df

,dt_nascimento_vitima,municipio_residencia,idade_vitima,raca_vitima,orient_sex_vitima,ident_gen_vitima,dt_notificacao,local_ocorrencia,num_envolvidos,tipo_violencia,viol_fisica,viol_sexual,viol_psicologica,lesao_autoprovocada,outras_vezes,autor_sexo
0,1937-11-02,Governador Valadares,72,Parda,Não preenchido,Não preenchido,2010-08-03,Residencia,Um,Física | Psicológica,Sim,Não,Sim,Não,Sim,Masculino
1,1992-04-30,Montes Claros,17,Parda,Não preenchido,Não preenchido,2010-02-07,Residencia,Um,Física | Psicológica,Sim,None,Sim,Não,Ignorado,Masculino
2,2003-04-07,Governador Valadares,7,Ignorado,Não preenchido,Não preenchido,2010-06-28,Ignorado,Um,Psicológica | Sexual,Não,Sim,Sim,Não,Ignorado,Masculino
3,1963-05-07,São José do Goiabal,46,Parda,Não preenchido,Não preenchido,2010-01-07,Residencia,Um,Não preenchido,Não,Não,Não,Não preenchido,Ignorado,Feminino
4,1917-08-01,Jaboticatubas,92,Branca,Não preenchido,Não preenchido,2010-05-20,Residencia,Dois ou mais,Não preenchido,Não,Não,Não,Não preenchido,Sim,Masculino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473902,1977-11-26,Carmo do Rio Claro,48,Branca,Heterossexual,Não se aplica,2025-11-24,Residencia,Um,Física | Psicológica,Sim,Não,Sim,Não,Não,Masculino
473903,1988-03-07,Passos,37,Parda,Ignorado,Ignorado,2025-12-03,Ignorado,Um,Física,Sim,Não,Não,Sim,Sim,Feminino
473904,1980-02-16,Itaú de Minas,45,Parda,Heterossexual,Ignorado,2025-11-23,Outro,Um,Física,Sim,Não,Não,Não,Não,Masculino
473905,1999-08-24,Passos,26,Parda,Ignorado,Ignorado,2025-12-02,Ignorado,Um,Física,Sim,Não,Não,Não,Ignorado,Masculino


In [25]:
df["raca_vitima"].value_counts()

raca_vitima
Parda             213837
Branca            160611
Preta              52409
Ignorado           35679
Não preenchido      5983
Amarela             3633
Indígena            1755
Name: count, dtype: Int64

# Modelagem

In [3]:
# cópia para manipular o dataframe
dfc = df.copy()

### Variável *dt_notificacao*

#### Extraindo data

In [4]:
# capturando ano, mês e dia
dfc["dia_notificacao"] = dfc["dt_notificacao"].dt.day
dfc["mes_notificacao"] = dfc["dt_notificacao"].dt.month
dfc["ano_notificacao"] = dfc["dt_notificacao"].dt.year

# separando o ano em outro dataframe
dfc_year = dfc["ano_notificacao"]

# removendo ele do dataframe cópia
dfc.drop(columns=["ano_notificacao"], inplace=True)

#### Variável *dia* (notificação)

- Segunda-feira = 0
- Terça-feira   = 1
- Quarta-feira  = 2
- Quinta-feira  = 3
- Sexta-feira   = 4
- Sábado        = 5
- Domingo       = 6

**Dia útil (segunda - quinta)**<br>
**Fim de semana (sexta - domingo)**

In [5]:
dfc["dia_semana"] = dfc["dt_notificacao"].dt.dayofweek
dfc["dia_util"] = dfc["dia_semana"].isin([0, 1, 2, 3]).astype(int)
dfc["fim_semana"] = dfc["dia_semana"].isin([4, 5, 6]).astype(int)

#### Variável mês (notificação)

- 1º trimestre: Jan-Mar
- 2º trimestre: Abr-Jun
- 3º trimestre: Jul-Set
- 4º trimestre: Out-Dez

In [6]:
# trimestre
dfc["primeiro_trim"] = dfc["mes_notificacao"].isin([1,2,3]).astype(int) 
dfc["segundo_trim"] = dfc["mes_notificacao"].isin([4,5,6]).astype(int) 
dfc["terceiro_trim"] = dfc["mes_notificacao"].isin([7,8,9]).astype(int) 
dfc["quarto_trim"] = dfc["mes_notificacao"].isin([10,11,12]).astype(int) 

# semestre (para testar se a performance melhora)
dfc["primeiro_semes"] = dfc["mes_notificacao"].isin([1,2,3,4,5,6]).astype(int) 
dfc["segundo_semes"] = dfc["mes_notificacao"].isin([7,8,9,10,11,12]).astype(int) 

### Variáveis de violência
- viol_fisica
- viol_sexual
- viol_psicologica
- lesao_autoprovocada (viol_autoprovocada)

#### Removendo registros onde todas as violências são valores nulos

In [7]:
dfc.rename(columns={"lesao_autoprovocada": "viol_autoprovocada"}, inplace=True)

violence_cols = ["viol_fisica", "viol_psicologica", "viol_sexual", "viol_autoprovocada"]

violence_categories = {
    "Sim": "1",
    "Não": "0",
    "Não preenchido": "0",
    "Ignorado": "0"
} 

for col in violence_cols:
    dfc[col] = dfc[col].fillna("0")
    dfc[col] = dfc[col].replace(violence_categories)

# transformando colunas para numéricas
dfc[violence_cols] = dfc[violence_cols].apply(pd.to_numeric, errors="coerce")

# removendo registros em que todos tipos de violência são 0
dfc = dfc[~((dfc["viol_fisica"] == 0) & 
            (dfc["viol_psicologica"] == 0) & 
            (dfc["viol_sexual"] == 0) & 
            (dfc["viol_autoprovocada"] == 0))]

### Variável *local_ocorrencia*

Para reduzir a dimensionalidade, a ideia é agrupar melhor as categorias:
- Residência
- Via pública
- Ignorado, Outro, Não preenchido -> **Outro**
- Bar ou similar, Local de pratica esportiva, Comercio/Serviços-> **Lazer/Consumo**
- Habitação coletiva, Escola, Indústria/Construção -> **Instituição pública/privada**

In [8]:
round((dfc["local_ocorrencia"].value_counts() / len(dfc)) * 100, 2)

local_ocorrencia
Residencia                    69.77
Via pública                    11.6
Ignorado                       9.18
Outro                          3.88
Bar ou similar                 1.89
Escola                         1.49
Comercio/Serviços              0.97
Habitação coletiva             0.53
Não preenchido                 0.38
Local de pratica esportiva     0.21
Industria/Construção            0.1
Name: count, dtype: Float64

In [9]:
def group_violence_place(place):
    if "Residencia" in place: return "Residência"

    if "Via pública" in place: return "Via pública"

    if any(x in place for x in ["Ignorado", "Outro", "Não preenchido"]): return "Outros"

    if any(x in place for x in ["Bar ou similar", "Local de pratica esportiva", "Comercio/Serviços"]): return "Lazer/Consumo"

    if any(x in place for x in ["Habitação coletiva", "Escola", "Industria/Construção"]): return "Instituição pública/privada"

dfc["local_ocorrencia"] = dfc["local_ocorrencia"].apply(group_violence_place)

In [12]:
dfc["local_ocorrencia"].value_counts()

local_ocorrencia
Residência                     312799
Outros                          60253
Via pública                     51999
Lazer/Consumo                   13799
Instituição pública/privada      9481
Name: count, dtype: int64

### Variáveis *raca_vitima*, *orient_sex_vitima*, *ident_gen_vitima*, *autor_sexo*, *outras_vezes* e *num_envolvidos* 

Essas variáveis terão o mesmo tratamento. Somente as categorias que podem agregar valor a análise serão utilizadas. Para cada variável, abaixo estão listadas as categorias que serão mantidas:

**raca_vitima**
- Parda
- Branca
- Preta
- Amarela
- Indígena

**orient_sex_vitima**
- Heterossexual
- Homossexual
- Bissexual

**ident_gen_vitima**
- Transsexual Mulher
- Transsexual Homem
- Travesti

**autor_sexo**
- Masculino
- Feminino
- Ambos os sexos

**outras_vezes**
- Sim
- Não

**num_envolvidos**
- Um
- Dois ou mais

In [13]:
# raca_vitima
dfc["vitima_cor_parda"] = np.where(dfc["raca_vitima"] == "Parda", 1, 0)
dfc["vitima_cor_branca"] = np.where(dfc["raca_vitima"] == "Branca", 1, 0)
dfc["vitima_cor_preta"] = np.where(dfc["raca_vitima"] == "Preta", 1, 0)
dfc["vitima_cor_amarela"] = np.where(dfc["raca_vitima"] == "Amarela", 1, 0)
dfc["vitima_cor_indigena"] = np.where(dfc["raca_vitima"] == "Indígena", 1, 0)

# orient_sex_Vitima e ident_gen_vitima
lgbt_condition = (dfc["orient_sex_vitima"].isin(["Homossexual", "Bissexual"])) | (dfc["ident_gen_vitima"].isin(["Transsexual Mulher", "Transsexual Homem", "Travesti"]))
dfc["vitima_lgbt"] =  np.where(lgbt_condition, 1, 0)

trans_condition = dfc["ident_gen_vitima"].isin(["Transsexual Mulher", "Transsexual Homem", "Travesti"])
dfc["vitima_cis_hetero"] =  np.where(
    (dfc["orient_sex_vitima"] == "Heterossexual") & 
    (~trans_condition), 1, 0)

# autor_sexo
dfc["autor_sex_masculino"] = np.where((dfc["autor_sexo"] == "Masculino") |
                                      (dfc["autor_sexo"] == "Ambos os sexos"), 1, 0)
dfc["autor_sex_feminino"] = np.where((dfc["autor_sexo"] == "Feminino") |
                                    (dfc["autor_sexo"] == "Ambos os sexos"), 1, 0)

# outras_vezes
dfc["violencia_repetitiva"] = np.where(dfc["outras_vezes"] == "Sim", 1, 0)
dfc["violencia_unica"] = np.where(dfc["outras_vezes"] == "Não", 1, 0)

# num_envolvidos
dfc["unico_agressor"] = np.where(dfc["num_envolvidos"] == "Um", 1, 0)

multiple_attackers_condition = (
    (dfc["num_envolvidos"] == "Dois ou mais") | 
    (dfc["autor_sexo"] == "Ambos os sexos")
)

dfc["multiplos_agressores"] = np.where(multiple_attackers_condition, 1, 0)

### Variável *idade_vitima*

Nesse caso, as idades serão tratadas por meio de faixas. Eu escolhi utilizar a classificação de faixa etária da OMS, que corresponde as seguintes categorias:

- Crianças (0-14)
- Jovens (15-24)
- Adultos (25-44)
- Meia-idade (45-59)
- Idosos (60-74)
- Seniores (75+) 

In [14]:
dfc["vitima_crianca"] = np.where((dfc["idade_vitima"] >= 0) & (dfc["idade_vitima"] <= 14), 1, 0)
dfc["vitima_jovem"] = np.where((dfc["idade_vitima"] >= 15) & (dfc["idade_vitima"] <= 24), 1, 0)
dfc["vitima_adulto"] = np.where((dfc["idade_vitima"] >= 25) & (dfc["idade_vitima"] <= 44), 1, 0)
dfc["vitima_meia_idade"] = np.where((dfc["idade_vitima"] >= 45) & (dfc["idade_vitima"] <= 59), 1, 0)
dfc["vitima_idoso"] = np.where((dfc["idade_vitima"] >= 60) & (dfc["idade_vitima"] <= 74), 1, 0)
dfc["vitima_senior"] = np.where((dfc["idade_vitima"] >= 75), 1, 0)

### Variável *municipio_residencia*

#### Usando regiões das cidades

In [15]:
# dataset de cidades de Minas Gerais (31) da API do IBGE
request_url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/31/municipios?view=nivelado"

df_mg_region = pd.read_json(request_url)
df_mg_region = df_mg_region[["municipio-nome", "mesorregiao-nome"]]
display(df_mg_region.head())
print(df_mg_region.shape)

,municipio-nome,mesorregiao-nome
0,Abadia dos Dourados,Triângulo Mineiro/Alto Paranaíba
1,Abaeté,Central Mineira
2,Abre Campo,Zona da Mata
3,Acaiaca,Zona da Mata
4,Açucena,Vale do Rio Doce


(853, 2)


#### Substituindo nomes de cidades da base do SES de acordo com a base do IBGE

In [16]:
cities_names_correct = {
    "Olhos-D'água": "Olhos-d'Água",
    "São João Del Rei": "São João del Rei",
    "MartinsSoares": "Martins Soares",
    "São Thomé das Letras": "São Tomé das Letras",
    "Pingo-D'água": "Pingo-d'Água",
    "Queluzita": "Queluzito",
    "Gouvêa": "Gouveia",
    "Passa-Vinte": "Passa Vinte",
    "Brasópolis": "Brazópolis",
    "Barão de Monte Alto": "Barão do Monte Alto"
}

dfc["municipio_residencia"] = df["municipio_residencia"].replace(cities_names_correct)

In [17]:
df_mg_region.rename(columns={"municipio-nome": "municipio_residencia", "mesorregiao-nome": "mesorregiao"}, inplace=True)

df_merged = pd.merge(
    dfc,
    df_mg_region,
    how="left",
    on="municipio_residencia"
)

### Removendo variáveis que não serão utilizadas

In [18]:
df_merged.columns

Index(['dt_nascimento_vitima', 'municipio_residencia', 'idade_vitima',
       'raca_vitima', 'orient_sex_vitima', 'ident_gen_vitima',
       'dt_notificacao', 'local_ocorrencia', 'num_envolvidos',
       'tipo_violencia', 'viol_fisica', 'viol_sexual', 'viol_psicologica',
       'viol_autoprovocada', 'outras_vezes', 'autor_sexo', 'dia_notificacao',
       'mes_notificacao', 'dia_semana', 'dia_util', 'fim_semana',
       'primeiro_trim', 'segundo_trim', 'terceiro_trim', 'quarto_trim',
       'primeiro_semes', 'segundo_semes', 'vitima_cor_parda',
       'vitima_cor_branca', 'vitima_cor_preta', 'vitima_cor_amarela',
       'vitima_cor_indigena', 'vitima_lgbt', 'vitima_cis_hetero',
       'autor_sex_masculino', 'autor_sex_feminino', 'violencia_repetitiva',
       'violencia_unica', 'unico_agressor', 'multiplos_agressores',
       'vitima_crianca', 'vitima_jovem', 'vitima_adulto', 'vitima_meia_idade',
       'vitima_idoso', 'vitima_senior', 'mesorregiao'],
      dtype='object')

In [19]:
df_merged.drop(columns=["dt_nascimento_vitima", "municipio_residencia", "idade_vitima", "raca_vitima", "orient_sex_vitima", "ident_gen_vitima", "dt_notificacao", "num_envolvidos", "outras_vezes", "tipo_violencia", "autor_sexo"], inplace=True)

df_merged.shape

(448331, 36)

In [20]:
df_merged.dtypes

local_ocorrencia        object
viol_fisica              int64
viol_sexual              int64
viol_psicologica         int64
viol_autoprovocada       Int64
dia_notificacao          int32
mes_notificacao          int32
dia_semana               int32
dia_util                 int64
fim_semana               int64
primeiro_trim            int64
segundo_trim             int64
terceiro_trim            int64
quarto_trim              int64
primeiro_semes           int64
segundo_semes            int64
vitima_cor_parda         int64
vitima_cor_branca        int64
vitima_cor_preta         int64
vitima_cor_amarela       int64
vitima_cor_indigena      int64
vitima_lgbt              int64
vitima_cis_hetero        int64
autor_sex_masculino      int64
autor_sex_feminino       int64
violencia_repetitiva     int64
violencia_unica          int64
unico_agressor           int64
multiplos_agressores     int64
vitima_crianca           int64
vitima_jovem             int64
vitima_adulto            int64
vitima_m

In [21]:
df_merged.to_parquet("dados_violencia_mulheres_ses_modelado.parquet")